In [282]:
import pandas as pd

In [283]:
dfranking = pd.read_csv("data/games/ranking.csv")
dfgamebox = pd.read_csv("data/games/games_details.csv")
dfgames = pd.read_csv("data/games/games.csv")

/tmp/ipykernel_12711/3784628109.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfgamebox = pd.read_csv("data/games/games_details.csv")


In [284]:
dfranking = dfranking.drop(columns=["LEAGUE_ID","SEASON_ID","CONFERENCE","HOME_RECORD","ROAD_RECORD","G","W","L","RETURNTOPLAY"])
dfranking["STANDINGSDATE"] = pd.to_datetime(dfranking["STANDINGSDATE"])

In [285]:
dfranking.head()

,TEAM_ID,STANDINGSDATE,TEAM,W_PCT
0,1610612743,2022-12-22,Denver,0.633
1,1610612763,2022-12-22,Memphis,0.633
2,1610612740,2022-12-22,New Orleans,0.613
3,1610612756,2022-12-22,Phoenix,0.594
4,1610612746,2022-12-22,LA Clippers,0.576


In [286]:
dfgames = dfgames[dfgames["SEASON"] > 2019]
dfgames = dfgames.drop(columns=["PTS_home","FG_PCT_home","FT_PCT_home","FG3_PCT_home","AST_home","REB_home","PTS_away","FG_PCT_away","FT_PCT_away","FT_PCT_away","FG3_PCT_away","AST_away","REB_away", "GAME_STATUS_TEXT"])
dfgames["GAME_DATE_EST"] = pd.to_datetime(dfgames["GAME_DATE_EST"])

In [287]:
dfgames.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,TEAM_ID_away,HOME_TEAM_WINS
0,2022-12-22,22200477,1610612740,1610612759,2022,1610612740,1610612759,1
1,2022-12-22,22200478,1610612762,1610612764,2022,1610612762,1610612764,1
2,2022-12-21,22200466,1610612739,1610612749,2022,1610612739,1610612749,1
3,2022-12-21,22200467,1610612755,1610612765,2022,1610612755,1610612765,1
4,2022-12-21,22200468,1610612737,1610612741,2022,1610612737,1610612741,0


In [288]:
dfranking = dfranking[dfranking["STANDINGSDATE"] >= dfgames["GAME_DATE_EST"].min()]

In [289]:
for s in dfgames["SEASON"].unique():
    first = dfgames[dfgames["SEASON"] == s]["GAME_DATE_EST"].min()
    last = first + pd.Timedelta(days=30)
    print(first,last)
    mask = ~dfranking["STANDINGSDATE"].between(first,last)
    dfranking = dfranking[mask]
    mask2 = ~dfgames["GAME_DATE_EST"].between(first,last)
    dfgames = dfgames[mask2]


2022-09-30 00:00:00 2022-10-30 00:00:00
2021-10-03 00:00:00 2021-11-02 00:00:00
2020-12-11 00:00:00 2021-01-10 00:00:00


In [290]:
for s in dfgames["SEASON"].unique():
    first = dfgames[dfgames["SEASON"] == s]["GAME_DATE_EST"].min()
    print(first)
    print(dfranking[dfranking["STANDINGSDATE"] == first])

2022-10-31 00:00:00
         TEAM_ID STANDINGSDATE           TEAM  W_PCT
780   1610612756    2022-10-31        Phoenix  0.833
781   1610612757    2022-10-31       Portland  0.833
782   1610612762    2022-10-31           Utah  0.750
783   1610612759    2022-10-31    San Antonio  0.714
784   1610612740    2022-10-31    New Orleans  0.667
785   1610612743    2022-10-31         Denver  0.571
786   1610612763    2022-10-31        Memphis  0.571
787   1610612750    2022-10-31      Minnesota  0.571
788   1610612742    2022-10-31         Dallas  0.500
789   1610612760    2022-10-31  Oklahoma City  0.500
790   1610612744    2022-10-31   Golden State  0.429
791   1610612746    2022-10-31    LA Clippers  0.429
792   1610612758    2022-10-31     Sacramento  0.333
793   1610612747    2022-10-31    L.A. Lakers  0.167
794   1610612745    2022-10-31        Houston  0.125
5055  1610612749    2022-10-31      Milwaukee  1.000
5056  1610612739    2022-10-31      Cleveland  0.833
5057  1610612738    2022-1

In [293]:
dfgames["HOME_TEAM_WIN_PCT"] = 0
dfgames["AWAY_TEAM_WIN_PCT"] = 0

In [294]:
for index, row in dfgames.iterrows():
    dfgames.at[index,"HOME_TEAM_WIN_PCT"] = dfranking[(dfranking["TEAM_ID"] == row["HOME_TEAM_ID"]) & (dfranking["STANDINGSDATE"] == row["GAME_DATE_EST"])]["W_PCT"].iloc[0]
    dfgames.at[index,"AWAY_TEAM_WIN_PCT"] = dfranking[(dfranking["TEAM_ID"] == row["VISITOR_TEAM_ID"]) & (dfranking["STANDINGSDATE"] == row["GAME_DATE_EST"])]["W_PCT"].iloc[0]


In [295]:
dfgames.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,TEAM_ID_away,HOME_TEAM_WINS,HOME_TEAM_WIN_PCT,AWAY_TEAM_WIN_PCT
0,2022-12-22,22200477,1610612740,1610612759,2022,1610612740,1610612759,1,0.613,0.323
1,2022-12-22,22200478,1610612762,1610612764,2022,1610612762,1610612764,1,0.543,0.364
2,2022-12-21,22200466,1610612739,1610612749,2022,1610612739,1610612749,1,0.667,0.710
3,2022-12-21,22200467,1610612755,1610612765,2022,1610612755,1610612765,1,0.600,0.235
4,2022-12-21,22200468,1610612737,1610612741,2022,1610612737,1610612741,0,0.500,0.419


In [299]:
dfgames.to_csv("dfgames.csv")
dfranking.to_csv("dfranking.csv")